<h1 align=center> Computer Vision: Assignment 2 </h1>

| [<img src="https://avatars0.githubusercontent.com/u/18689888" width="150px;" height="150px;"/><br /><sub><b>Amr M. Kayid</b></sub>](https://github.com/AmrMKayid)| [<img src="https://avatars2.githubusercontent.com/u/15708438" width="150px;" height="150px;"/><br /><sub><b>Abdullah Elkady</b></sub>](https://github.com/AbdullahKady) |
| :---: | :---: | 
| **37-15594** | **37-16401** |
| **T10** | **T10** |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
original_image = plt.imread("GUC.jpg")
plt.imshow(original_image, cmap="gray")

In [ ]:
def get_thresholds(image, N):
    """
    N is the number of thresholds, thus should result in n + 1 segments/colors
    
    Returns a list of thresholds (size N).
    """
    
    thresholds = [n * (255 / (N+1)) for n in range(N+2)] # 2 extra bounds (0&255)
    old_thresholds = [0, *[None]*N, 255]
    
    while thresholds != old_thresholds:
        threshold_tuples = list(zip(thresholds[:-2], thresholds[1:-1], thresholds[2:]))
        for i, (t1, t2, t3) in enumerate(threshold_tuples):
            mean_1 = image[(image >= t1) & (image < t2)].mean()
            mean_2 = image[(image >= t2) & (image < t3)].mean()
            old_thresholds[i+1] = thresholds[i+1] # To avoid reference comparision :(
            thresholds[i+1] = (mean_1+mean_2)/2
    return thresholds # Exclude the ones used for calculations (0&255)

In [ ]:
def threshold_image(image, threshold):
    # Thresholds (binarizes) 'image' according to 'threshold'? Duhh! \_0_/
    result = np.zeros_like(image)
    result[image >= threshold] = 255
    result[image < threshold] = 0
    return result

In [ ]:
def segment_image(image, N):
    """
    N is the number of thresholds, thus should result in n + 1 segments/colors

    Returns a tuple (thresholds, binary_images, gray_scale_image)
    Where:
        - thresholds: a list of the thresholds (size=N)
        - binary_images: a list of images of each segment (size=N)
        - gray_scale_image: a single image, with (N+1) colors representing the segments
    """
    
    thresholds = get_thresholds(image, N)
    binary_images = [threshold_image(image, threshold) for threshold in thresholds[1:-1]] # Ignore the bounds(0&255)
    shades = [n * (255 / (N)) for n in range(N+1)] # Colors as far as possible (according to ratio 255/(N+1))
    gray_scale_image = np.copy(image)
    for shade, (t1, t2) in zip(shades, zip(thresholds[:-1], thresholds[1: ])):
        gray_scale_image[(image>=t1) & (image<t2)] = shade
    return thresholds[1:-1], binary_images, gray_scale_image

In [ ]:
def save_image(fname, arr, vmin=None, vmax=None, cmap='gray', format=None, origin=None):
    # Shamelessly stolen from matplotlib svn source code, to avoid using Pillow:
    # https://stackoverflow.com/a/978306/7502260 (Save image without axes)
    from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
    from matplotlib.figure import Figure

    fig = Figure(figsize=arr.shape[::-1], dpi=1, frameon=False)
    canvas = FigureCanvas(fig)
    fig.figimage(arr, cmap=cmap, vmin=vmin, vmax=vmax, origin=origin)
    fig.savefig(fname, dpi=1, format=format)

In [ ]:
def save_output(thresholds, b_images, g_image):
    # Saves to the file-system according to the assignment requirements.
    thresholds_count = len(thresholds)
    with open('Thresholds_{}.txt'.format(thresholds_count), 'w') as text_file:
        text_file.write(','.join([str(t) for t in thresholds]))

    for i, b_img in enumerate(b_images):
        save_image('GUC_{}_{}.jpg'.format(thresholds_count, i+1), b_img)
    save_image('GUC_{}.jpg'.format(thresholds_count), g_image)

In [ ]:
save_output(*segment_image(original_image, 3))

In [ ]:
save_output(*segment_image(original_image, 4))